<a href="https://colab.research.google.com/github/eirik-dn/NLP-For-Finance-Final-Assignment/blob/main/NLP_For_Finance_Final_Assignment_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# NLP For Finance Final Assignment
# BoW Code

**Name: Eirik Dalen Nyrén**

**Student ID:102060**

---



**Prepatory Steps:**


*   Import packages
*   Download Data



In [1]:

!pip install transformers datasets[torch] nltk nlpaug scikit-learn torch seaborn pdfplumber pdfminer.six linearmodels swifter
!pip uninstall -y numpy gensim scipy smart-open
!pip install numpy==1.25.2 scipy gensim smart-open
!pip install tdqm
!apt install -y nvidia-cuda-toolkit
!pip install llama-cpp-python'==0.2.53' --force-reinstall --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
 

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=76d2ecbf14e58a3d5ebd4e1dbe1915287945419749fabf73dab921f57245076c
  Stored in directory: /root/.cache/pip/wheels/c8/c7/30/e5935be2cfa6883be72462333edc414d8928f3c78eaabec38a
Successfully built tdqm
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C
^C
^C


In [1]:
# import packages

# Core packages
import os
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# NLTK for text preprocessing
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment.util import mark_negation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import ngrams
# function for preproscessing
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

# spacy for text preprocessing
import spacy

# Sklearn for vectorization, similarity metrics, clustering
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Gensim for word embeddings
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, KeyedVectors, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

# Statsmodels for statistical modeling
import statsmodels.api as sm
import statsmodels.formula.api as smf

# swifter
import swifter

# import tdqm to look at progression of loops
from tqdm import tqdm
tqdm.pandas()

# import sql
import sqlite3
conn = sqlite3.connect('LLMs_results.db')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# @title Load the textual data for the Earnings Calls of S&P500 from 2015 to 2021
import pandas as pd

# Load the Sample of Earnings Calls for the S&P500 from 2015 to 2021 with financials
Sample_EC = pd.read_csv("https://www.dropbox.com/scl/fi/2p7ahxroqj9pwf98ni5an/Sample_Calls.csv?rlkey=zfieicvz891u4e3z0aroeg0u7&dl=1")

# Load the Sample's Presentation texts
Sample_Presentations = pd.read_feather("https://www.dropbox.com/scl/fi/uceh2xva5g4apbmt92cgt/Sample_Calls_Presentations.feather?rlkey=ln4nzsa4nenqyvm0pg2cur9sp&dl=1")

# Load the Q&A session textual data for the sample
Sample_QAs = pd.read_feather("https://www.dropbox.com/scl/fi/iq4111nlmsykp2tzxk9xg/Sample_Calls_QA.feather?rlkey=xabjqmwhesx05jivrlfzkgj6m&dl=1")


## Note: column "file_name" is to be used for the merging

# ## Columns:
#     # Identifiers
#     "GVKEY": "A unique company identifier used by Compustat.",
#     "date_rdq": "The reporting date of the quarterly earnings or a related key event date.",
#     "co_conm": "The company’s name in CRSP.",

#     # Earnings Call Columns
#     "file_name": "The identifier or filename of the earnings call transcript.",
#     "CAR-11-Carhart": "Cumulative Abnormal Return over an event window using the Carhart 4-factor model.",
#     "CAR-11-ff3": "Cumulative Abnormal Return over an event window using the Fama-French 3-factor model.",
#     "CAR01-Carhart": "Cumulative Abnormal Return (alternative window) using the Carhart 4-factor model.",
#     "CAR01-ff3": "Cumulative Abnormal Return (alternative window) using the Fama-French 3-factor model.",
#     "IV": "Implied volatility (often from options) reflecting expected future stock price volatility.",
#     "hvol": "Historical volatility of the stock, based on past price movements.",
#     "IV_l1d": "Implied volatility lagged by one day.",
#     "IV_l2d": "Implied volatility lagged by two days.",
#     "IV_f1d": "Implied volatility forecasted or measured one day forward.",

#     # I/B/E/S Columns
#     "NUMEST": "The number of analyst estimates contributing to the consensus.",
#     "NUMUP": "The number of analysts who have revised their EPS estimates upward.",
#     "NUMDOWN": "The number of analysts who have revised their EPS estimates downward.",
#     "MEDEST": "The median of analyst EPS estimates.",
#     "MEANEST": "The mean of analyst EPS estimates.",
#     "ACTUAL": "The I/B/E/S standardized actual EPS figure, often adjusted for comparability.",
#     "surp": "The earnings surprise, typically ACTUAL minus MEANEST.",
#     "SurpDec": "A scaled or decimalized version of the earnings surprise.",

#     # Compustat Columns
#     "atq": "Total Assets (Quarterly)",
#     "actq": "Current Assets (Quarterly)",
#     "cheq": "Cash and Cash Equivalents (Quarterly)",
#     "rectq": "Accounts Receivable (Quarterly)",
#     "invtq": "Inventory (Quarterly)",
#     "ltq": "Total Liabilities (Quarterly)",
#     "lctq": "Current Liabilities (Quarterly)",
#     "apq": "Accounts Payable (Quarterly)",
#     "ceqq": "Total Equity (Quarterly)",
#     "seqq": "Common Equity (Quarterly)",

#     "capxy": "Capital Expenditures (Note: 'capxy' is annual by default, quarterly approximations derived from segments)",
#     "dpq": "Depreciation and Amortization (Quarterly)",
#     "saleq": "Revenue (Quarterly)",
#     "cogsq": "Cost of Goods Sold (Quarterly)",
#     "oiadpq": "Operating Income (Quarterly)",
#     "niq": "Net Income (Quarterly)",
#     "epspxq": "Basic Earnings Per Share (Quarterly)",
#     "epspiq": "Diluted Earnings Per Share (Quarterly)",
#     "dlttq": "Long-Term Debt (Quarterly)",
#     "dlcq": "Debt in Current Liabilities (Quarterly)",
#     "prccq": "Price Close - Fiscal Quarter",
#     "cshoq": "Common Shares Outstanding (Quarterly)",
#     "dvpq": "Dividends Paid (Quarterly)",
#     "xintq": "Interest Expense (Quarterly)"


In [3]:
# import the Loughran-McDonald_MasterDictionary
github_LM_dict_url = "https://raw.githubusercontent.com/eirik-dn/NLP-For-Finance-Final-Assignment/refs/heads/main/Loughran-McDonald_MasterDictionary_1993-2024.csv"
LM_dict_global            = pd.read_csv(github_LM_dict_url)
LM_dict_global.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,755,2.955070e-08,1.945421e-08,4.078069e-06,140,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.174200e-10,8.060019e-12,8.919011e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.522600e-10,1.089343e-10,5.105359e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.135060e-09,6.197922e-10,1.539279e-07,28,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,9620,3.765268e-07,3.825261e-07,3.421836e-05,1295,0,0,0,0,0,0,0,0,3,12of12inf


In [4]:
# import the harvard dictonary¨
HV=pd.read_csv("https://www.dropbox.com/s/wjucnpw39uuxupf/HarvardGI4.txt?dl=1", delimiter='\t',low_memory=False)
HV.head()


,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined\
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|\
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|\
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|\
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,\


In [5]:
# extract the answers from Sample_QA
temp = Sample_QAs[Sample_QAs['QA'] == "a"]

# Generate Earnings call level data by aggregating in the file_name level
temp = temp.groupby('file_name')['QA_text'].apply(lambda x: ''.join(x)).reset_index()

# Merge with Sample_EC and name it as Sample_Answers
Sample_answers_global = Sample_EC.merge(temp)
Sample_answers_global.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,niq,epspxq,epspiq,dlttq,dlcq,prccq,cshoq,dvpq,xintq,QA_text
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,1610.0,0.99,0.99,37328.0,517.0,61.91,1628.542,0.0,245.0,"Jeff, this is Rick. I will cover the first and..."
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,1354.0,0.83,0.83,29490.0,2423.0,57.12,1617.359,0.0,215.0,"Okay. Hi, Jeff, it's Rick. So I'll take I gues..."
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,1598.0,0.97,0.97,37284.0,26.0,63.07,1624.908,0.0,271.0,"Sure, Jami; this is Rick. Thank you for the qu..."
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,1391.0,0.85,0.85,36440.0,402.0,62.62,1592.513,0.0,277.0,"Hi, Jami; it's Bill. So on your operating marg..."
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,1711.0,1.07,1.07,36526.0,425.0,65.16,1591.366,0.0,273.0,"Yes. Well, first, thanks for the question. May..."


In [6]:
# Merge EC and presentations
EC_pres_global = pd.merge(
    Sample_EC,
    Sample_Presentations[['file_name','presentation']],
    on = 'file_name',
    how = 'outer'
)

EC_pres_global.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,niq,epspxq,epspiq,dlttq,dlcq,prccq,cshoq,dvpq,xintq,presentation
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,1336.0,2.97,2.97,22554.0,9326.0,72.22,443.146,0.0,325.0,"Ladies and gentlemen, thank you for standi..."
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,921.0,2.06,2.06,21080.0,8011.0,71.34,438.739,0.0,357.0,Welcome to the Prudential quarterly earnin...
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,1827.0,4.14,4.14,21480.0,7737.0,81.65,431.704,0.0,314.0,"Ladies and gentlemen, thank you for standi..."
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,440.1,0.42,0.42,7477.6,648.3,72.01,1103.837,0.0,56.8,"Ladies and gentlemen, thank you for standi..."
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,747.7,0.71,0.71,8685.5,645.8,78.75,1103.843,0.0,57.2,"Ladies and gentlemen, thank you for standi..."


In [7]:
# merge the presentation and answers
merged_ec_global = pd.merge(
    EC_pres_global,
    Sample_answers_global[['file_name', 'QA_text']],
    on='file_name',
    how='outer'
)

# Display the merged DataFrame's first few rows to verify the result
merged_ec_global.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,epspxq,epspiq,dlttq,dlcq,prccq,cshoq,dvpq,xintq,presentation,QA_text
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,2.97,2.97,22554.0,9326.0,72.22,443.146,0.0,325.0,"Ladies and gentlemen, thank you for standi...",Sure. So in terms of sales on the Life Planner...
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,2.06,2.06,21080.0,8011.0,71.34,438.739,0.0,357.0,Welcome to the Prudential quarterly earnin...,"Ryan, this is Steve. We've long encouraged peo..."
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,4.14,4.14,21480.0,7737.0,81.65,431.704,0.0,314.0,"Ladies and gentlemen, thank you for standi...","Jimmy, it's Rob. I'll take the first of those ..."
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,0.42,0.42,7477.6,648.3,72.01,1103.837,0.0,56.8,"Ladies and gentlemen, thank you for standi...","Great, Gregg, thank you for the questions. Dav..."
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,0.71,0.71,8685.5,645.8,78.75,1103.843,0.0,57.2,"Ladies and gentlemen, thank you for standi...","Great, Mike, thanks for the question. For the ..."


# Part 1 (Bag-of-Words Methods):


**Goal:** Performe sentiment analysis on both the presentation and QA (with emphasis on answers) using the Loughran-McDonald (LM) Dictionary and Harvard dictionary seperatly and potenially expand the dictonary using Word2Vec or Doc2Vec. Approach 1 will not account for negations, while Approach 2 will account for negations.


**Steps:**


*   Step 1: Preprocess text by turning into lowercase. Extract the negative and positive words from the Loughran-Mcdonald dictionary

*   Step 2: Count number of positive and negative words seperatly in both the presentation and the QA

*   Step 3: Create sentiment score by normalizing the scores by doing: Sentiment Score = (pos count - neg_count) / total words. Here total words is only alphabetic words.

*   Step 4: Create correlation matrix and performe regression




---
# Approach 1: Not accounting for negations
---

**Step 1:** Extract the negative and positive words from the LM

In [9]:
# import random package
#import random
#sample_size = 3
#sample_file_names = random

# This assumes 'file_name' is the column containing your file names.
#sample_file_names = random.sample(list(EC_pres_global['file_name'].unique()), sample_size)

# Create subsets for both DataFrames using the same file names
#EC_pres_BoW = EC_pres_global[EC_pres_global['file_name'].isin(sample_file_names)].copy()
#answers_BoW  = Sample_answers_global[Sample_answers_global['file_name'].isin(sample_file_names)].copy()
EC_pres_BoW = EC_pres_global.copy()
answers_BoW  = Sample_answers_global.copy()

# The LM dict remains the same, just copying the global version
LM_dict_BoW = LM_dict_global.copy()


**Perform preprocessing**


In [10]:
# function for preproscessing
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english")) - {"not", "no"}
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
  #if isinstance(text,list):
   # text = " ".join(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# takes approximatly 2 minutes to run
# use tdqm
tqdm.pandas()

# preprocess the text that for not negations
EC_pres_BoW['presentation_clean_non_neg'] = EC_pres_BoW['presentation'].progress_apply(preprocess_text)

# preprocess the text not for negations
answers_BoW['QA_text_clean_non_neg']      = answers_BoW['QA_text'].progress_apply(preprocess_text)

# preprocess text for negations
EC_pres_BoW['presentation_clean_neg']     = EC_pres_BoW['presentation'].str.lower()

# preprocess answers for negations
answers_BoW['QA_text_clean_neg']          = answers_BoW['QA_text'].str.lower()

100%|██████████| 2877/2877 [01:12<00:00, 39.64it/s]


In [12]:
# create functio to preprocess dictionaries
def preprocess_dict_word(word):
    if pd.isna(word):
        return word  # Handle NaN
    return lemmatizer.lemmatize(word.lower())  # Lowercase + lemmatize

In [13]:
# turn the LM "Word" column into lowercase
LM_dict_BoW['Word'] = LM_dict_BoW['Word'].progress_apply(preprocess_dict_word)

# Extract positive words
LM_pos_BoW = LM_dict_BoW[LM_dict_BoW['Positive'] > 0]['Word'].tolist()

# Extract negative words
LM_neg_BoW = LM_dict_BoW[LM_dict_BoW['Negative'] > 1]['Word'].tolist()

100%|██████████| 86553/86553 [00:00<00:00, 156452.31it/s]


In [14]:
# take a look at positive words
LM_pos_BoW[:10]

['able',
 'abundance',
 'abundant',
 'acclaimed',
 'accomplish',
 'accomplished',
 'accomplishes',
 'accomplishing',
 'accomplishment',
 'accomplishment']

In [15]:
# take a look at negative words
LM_neg_BoW[:10]

['abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abandonment',
 'abandon',
 'abdicated',
 'abdicates',
 'abdicating',
 'abdication']

**Step 2:** Count number of positive and negative words seperatly in both the presentation and the QA for LM

In [16]:
# download nltk punkt
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [17]:
# takes approximatly 3.30 min to run
# create dicts to store counts
LM_pos_Bow_count_pres   = []
LM_neg_BoW_count_pres   = []
LM_sentiment_score_pres  = []

#loop over the presentations to find positive and negative words
for words in tqdm(EC_pres_BoW['presentation_clean_non_neg'], total = len(EC_pres_BoW['presentation_clean_non_neg']), desc = "Processing Presentations"):

  pos_count_lm_BoW  = sum(word in LM_pos_BoW for word in words)
  neg_count_lm_BoW  = sum(word in LM_neg_BoW for word in words)

  LM_pos_Bow_count_pres.append(pos_count_lm_BoW)
  LM_neg_BoW_count_pres.append(neg_count_lm_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_lm_BoW - neg_count_lm_BoW) / total_words if total_words > 0 else 0
  LM_sentiment_score_pres.append(sentiment_score)

EC_pres_BoW['LM_pos_count']              = LM_pos_Bow_count_pres
EC_pres_BoW['LM_neg_count']              = LM_neg_BoW_count_pres
EC_pres_BoW['sentiment_score_pres_LM']   = LM_sentiment_score_pres


Processing Presentations: 100%|██████████| 2877/2877 [03:35<00:00, 13.34it/s]


In [18]:
# take allok at the sentiment scores for presenations
EC_pres_BoW[['sentiment_score_pres_LM']].head()

,sentiment_score_pres_LM
0,0.001265
1,0.003831
2,0.018987
3,0.002293
4,0.012982


In [19]:
# takes approximatly 3.30 min to run
# create dicts to store counts
LM_pos_Bow_count_answers        = []
LM_neg_BoW_count_answers        = []
LM_sentiment_score_answers      = []

#loop over the answers to find positive and negative words
for words in tqdm(answers_BoW['QA_text_clean_non_neg'], total=len(answers_BoW['QA_text_clean_non_neg']), desc="Processing Answers"):

  pos_count_lm_BoW  = sum(word in LM_pos_BoW for word in words)
  neg_count_lm_BoW  = sum(word in LM_neg_BoW for word in words)

  LM_pos_Bow_count_answers.append(pos_count_lm_BoW)
  LM_neg_BoW_count_answers.append(neg_count_lm_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_lm_BoW - neg_count_lm_BoW) / total_words if total_words > 0 else 0
  LM_sentiment_score_answers.append(sentiment_score)

answers_BoW['LM_pos_count']               = LM_pos_Bow_count_answers
answers_BoW['LM_neg_count']               = LM_neg_BoW_count_answers
answers_BoW['sentiment_score_answers_LM'] = LM_sentiment_score_answers

Processing Answers: 100%|██████████| 2877/2877 [03:36<00:00, 13.30it/s]


In [20]:
# take a look at sentiment scores for answers
answers_BoW['sentiment_score_answers_LM'].head()

,sentiment_score_answers_LM
0,0.028535
1,0.018055
2,0.019081
3,0.002985
4,0.015055


**Now for the Harvard Dictonary**

In [21]:
# preprocess harvard dictionary
HV['Entry'] = HV['Entry'].progress_apply(preprocess_dict_word)

100%|██████████| 11789/11789 [00:00<00:00, 156454.46it/s]


In [22]:
# Extract positive and negative words
HV_pos_BoW = HV[~HV['Positiv'].isna()]['Entry'].to_list()
HV_neg_BoW = HV[~HV['Negativ'].isna()]['Entry'].to_list()

print(HV_pos_BoW)
print(HV_neg_BoW)

['abide', 'ability', 'able', 'abound', 'absolve', 'absorbent', 'absorption', 'abundance', 'abundant', 'accede', 'accentuate', 'accept', 'acceptable', 'acceptance', 'accessible', 'accession', 'acclaim', 'acclamation', 'accolade', 'accommodate', 'accommodation', 'accompaniment', 'accomplish', 'accomplishment', 'accord#2', 'accord#3', 'accord#5', 'accordance', 'accountable', 'accrue', 'accuracy', 'accurate', 'accurateness', 'achieve', 'achievement', 'acknowledgement', 'acquaint', 'acquaintance', 'acquit', 'acquittal', 'actual#1', 'actual#2', 'actuality', 'adamant', 'adaptability', 'adaptable', 'adaptation', 'adaptive', 'adept', 'adeptness', 'adequate', 'adherence', 'adherent', 'adhesion', 'adhesive', 'adjunct', 'adjust#2', 'adjustable', 'adjustment', 'admirable', 'admiration', 'admire', 'admirer', 'admit#1', 'admit#2', 'admit#3', 'admittance', 'adorable', 'adore', 'adorn', 'adornment', 'adroit', 'adroitly', 'adulation', 'adult#2', 'advance#1', 'advance#2', 'advance#3', 'advancement', 'adv

In [23]:
# takes approximatly 6min to run
# create dicts to store counts
HV_pos_Bow_count_pres   = []
HV_neg_BoW_count_pres   = []
HV_sentiment_score_pres  = []

#loop over the presentations to find positive and negative words
for words in tqdm(EC_pres_BoW['presentation_clean_non_neg'], total = len(EC_pres_BoW['presentation_clean_non_neg']), desc = "Processing Presentations"):

  pos_count_HV_BoW  = sum(word in HV_pos_BoW for word in words)
  neg_count_HV_BoW  = sum(word in HV_neg_BoW for word in words)

  HV_pos_Bow_count_pres.append(pos_count_HV_BoW)
  HV_neg_BoW_count_pres.append(neg_count_HV_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_HV_BoW - neg_count_HV_BoW) / total_words if total_words > 0 else 0
  HV_sentiment_score_pres.append(sentiment_score)

EC_pres_BoW['HV_pos_count']              = HV_pos_Bow_count_pres
EC_pres_BoW['HV_neg_count']              = HV_neg_BoW_count_pres
EC_pres_BoW['sentiment_score_pres_HV']   = HV_sentiment_score_pres


Processing Presentations: 100%|██████████| 2877/2877 [05:57<00:00,  8.05it/s]


In [24]:
# take a look
EC_pres_BoW.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,xintq,presentation,presentation_clean_non_neg,presentation_clean_neg,LM_pos_count,LM_neg_count,sentiment_score_pres_LM,HV_pos_count,HV_neg_count,sentiment_score_pres_HV
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,325.0,"Ladies and gentlemen, thank you for standi...","[lady, gentleman, thank, standing, welcome, fi...","ladies and gentlemen, thank you for standi...",59,56,0.001265,115,30,0.035850
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,357.0,Welcome to the Prudential quarterly earnin...,"[welcome, prudential, quarterly, earnings, cal...",welcome to the prudential quarterly earnin...,70,61,0.003831,114,34,0.034057
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,314.0,"Ladies and gentlemen, thank you for standi...","[lady, gentleman, thank, standing, welcome, pr...","ladies and gentlemen, thank you for standi...",98,53,0.018987,128,22,0.044726
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,56.8,"Ladies and gentlemen, thank you for standi...","[lady, gentleman, thank, standing, welcome, q,...","ladies and gentlemen, thank you for standi...",52,46,0.002293,101,40,0.023309
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,57.2,"Ladies and gentlemen, thank you for standi...","[lady, gentleman, thank, standing, welcome, q,...","ladies and gentlemen, thank you for standi...",78,41,0.012982,113,46,0.023509


In [25]:
# takes approximtley 6.30 min to run
# create dicts to store counts for HV
HV_pos_BoW_count_answers        = []
HV_neg_BoW_count_answers        = []
HV_sentiment_score_answers      = []

#loop over the answers to find positive and negative words
for words in tqdm(answers_BoW['QA_text_clean_non_neg'], total=len(answers_BoW['QA_text_clean_non_neg']), desc="Processing Answers"):

  pos_count_HV_BoW  = sum(word in HV_pos_BoW for word in words)
  neg_count_HV_BoW  = sum(word in HV_neg_BoW for word in words)

  HV_pos_BoW_count_answers.append(pos_count_HV_BoW)
  HV_neg_BoW_count_answers.append(neg_count_HV_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_HV_BoW - neg_count_HV_BoW) / total_words if total_words > 0 else 0
  HV_sentiment_score_answers.append(sentiment_score)

answers_BoW['HV_pos_count']               = HV_pos_BoW_count_answers
answers_BoW['HV_neg_count']               = HV_neg_BoW_count_answers
answers_BoW['sentiment_score_answers_HV'] = HV_sentiment_score_answers

Processing Answers: 100%|██████████| 2877/2877 [06:21<00:00,  7.53it/s]


---
# Approach 2: Accounting for negations
---


**First for the LM Dictionary**

In [26]:
# preprocess the LM dict for negations
LM_dict_neg         = LM_dict_global.copy()
LM_dict_neg['Word'] = LM_dict_neg['Word'].str.lower()

# extract the positive and negatiw words
LM_pos_neg_BoW  = LM_dict_neg[LM_dict_neg['Positive'] > 0]['Word'].tolist()
LM_neg_neg_BoW  = LM_dict_neg[LM_dict_neg['Negative'] > 1]['Word'].tolist()

In [27]:
# takes approximatly 8 min to run
# inialize the lists
LM_pos_Bow_count_pres_neg   = []
LM_neg_BoW_count_pres_neg   = []
LM_sentiment_score_pres_neg    = []

#loop over the presentations to find positive and negative words
for text in tqdm(EC_pres_BoW['presentation_clean_neg'], total=len(EC_pres_BoW['presentation_clean_neg']), desc = "Processing presentations with negations"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in LM_pos_neg_BoW:
        neg_count += 1
      elif base_word in LM_neg_neg_BoW:
        pos_count += 1
    else:
      if word in LM_pos_neg_BoW:
        pos_count += 1
      elif word in LM_neg_neg_BoW:
        neg_count += 1

  LM_pos_Bow_count_pres_neg.append(pos_count)
  LM_neg_BoW_count_pres_neg.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  LM_sentiment_score_pres_neg.append(sentiment_score)

EC_pres_BoW['LM_pos_count_neg']              = LM_pos_Bow_count_pres_neg
EC_pres_BoW['LM_neg_count_neg']              = LM_neg_BoW_count_pres_neg
EC_pres_BoW['sentiment_score_pres_neg_LM']   = LM_sentiment_score_pres_neg

Processing presentations with negations: 100%|██████████| 2877/2877 [08:03<00:00,  5.96it/s]


In [28]:
# take a look
EC_pres_BoW.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,presentation_clean_neg,LM_pos_count,LM_neg_count,sentiment_score_pres_LM,HV_pos_count,HV_neg_count,sentiment_score_pres_HV,LM_pos_count_neg,LM_neg_count_neg,sentiment_score_pres_neg_LM
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,"ladies and gentlemen, thank you for standi...",59,56,0.001265,115,30,0.035850,59,48,0.002751
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,welcome to the prudential quarterly earnin...,70,61,0.003831,114,34,0.034057,69,52,0.004250
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,"ladies and gentlemen, thank you for standi...",98,53,0.018987,128,22,0.044726,98,40,0.014841
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,"ladies and gentlemen, thank you for standi...",52,46,0.002293,101,40,0.023309,52,43,0.002099
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,"ladies and gentlemen, thank you for standi...",78,41,0.012982,113,46,0.023509,74,40,0.007197


In [29]:
# takes approximatly 10 min to run
# intilze the lists
LM_pos_Bow_count_answers_neg        = []
LM_neg_BoW_count_answers_neg        = []
LM_sentiment_score_answers_neg      = []

#loop over the answers to find positive and negative words
for text in tqdm(answers_BoW['QA_text'], total=len(answers_BoW['QA_text']), desc = "Processing Answers"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in LM_pos_neg_BoW:
        neg_count += 1
      elif base_word in LM_neg_neg_BoW:
        pos_count += 1
    else:
      if word in LM_pos_neg_BoW:
        pos_count += 1
      elif word in LM_neg_neg_BoW:
        neg_count += 1

  LM_pos_Bow_count_answers_neg.append(pos_count)
  LM_neg_BoW_count_answers_neg.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  LM_sentiment_score_answers_neg.append(sentiment_score)

answers_BoW['LM_pos_count_neg']               = LM_pos_Bow_count_answers_neg
answers_BoW['LM_neg_count_neg']               = LM_neg_BoW_count_answers_neg
answers_BoW['sentiment_score_answers_neg_LM'] = LM_sentiment_score_answers_neg

Processing Answers: 100%|██████████| 2877/2877 [09:49<00:00,  4.88it/s]


In [30]:
# take a look
answers_BoW.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,QA_text_clean_neg,LM_pos_count,LM_neg_count,sentiment_score_answers_LM,HV_pos_count,HV_neg_count,sentiment_score_answers_HV,LM_pos_count_neg,LM_neg_count_neg,sentiment_score_answers_neg_LM
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,"jeff, this is rick. i will cover the first and...",99,32,0.028535,137,30,0.045571,97,30,0.013291
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,"okay. hi, jeff, it's rick. so i'll take i gues...",82,33,0.018055,128,36,0.033898,80,30,0.008449
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,"sure, jami; this is rick. thank you for the qu...",60,16,0.019081,94,29,0.028187,58,16,0.008533
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,"hi, jami; it's bill. so on your operating marg...",46,39,0.002985,88,45,0.018337,45,36,0.001823
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,"yes. well, first, thanks for the question. may...",97,52,0.015055,135,44,0.030445,99,44,0.009048


**Now for the HV Dictionary**

In [31]:
# preprocess the HV dictionary for negations
HV_neg         = HV.copy()
HV_neg['Entry'] = HV_neg['Entry'].str.lower()

# extract the positive and negatiw words
HV_pos_neg_BoW  = HV_neg[~HV_neg['Positiv'].isna()]['Entry'].tolist()
HV_neg_neg_BoW  = HV_neg[~HV_neg['Negativ'].isna()]['Entry'].tolist()

In [32]:
# takes approximatley 13 min to run
# inialize the lists
HV_pos_Bow_count_pres_neg   = []
HV_neg_BoW_count_pres_neg   = []
HV_sentiment_score_pres_neg = []

#loop over the presentations to find positive and negative words
for text in tqdm(EC_pres_BoW['presentation_clean_neg'], total=len(EC_pres_BoW['presentation_clean_neg']), desc = "Processing presentations with negations"):

  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in HV_pos_neg_BoW:
        neg_count += 1
      elif base_word in HV_neg_neg_BoW:
        pos_count += 1
    else:
      if word in HV_pos_neg_BoW:
        pos_count += 1
      elif word in HV_neg_neg_BoW:
        neg_count += 1

  HV_pos_Bow_count_pres_neg.append(pos_count)
  HV_neg_BoW_count_pres_neg.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  HV_sentiment_score_pres_neg.append(sentiment_score)

EC_pres_BoW['HV_pos_count_neg']              = HV_pos_Bow_count_pres_neg
EC_pres_BoW['HV_neg_count_neg']              = HV_neg_BoW_count_pres_neg
EC_pres_BoW['sentiment_score_pres_neg_HV']   = HV_sentiment_score_pres_neg

Processing presentations with negations: 100%|██████████| 2877/2877 [12:11<00:00,  3.93it/s]


In [33]:
# takes apporximatley 15 min to run
# intilze the lists
HV_pos_Bow_count_answers_neg        = []
HV_neg_BoW_count_answers_neg        = []
HV_sentiment_score_answers_neg      = []

#loop over the answers to find positive and negative words
for text in tqdm(answers_BoW['QA_text'], total=len(answers_BoW['QA_text']), desc = "Processing Answers"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in HV_pos_neg_BoW:
        neg_count += 1
      elif base_word in HV_neg_neg_BoW:
        pos_count += 1
    else:
      if word in HV_pos_neg_BoW:
        pos_count += 1
      elif word in HV_neg_neg_BoW:
        neg_count += 1

  HV_pos_Bow_count_answers_neg.append(pos_count)
  HV_neg_BoW_count_answers_neg.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  HV_sentiment_score_answers_neg.append(sentiment_score)

answers_BoW['HV_pos_count_neg']               = HV_pos_Bow_count_answers_neg
answers_BoW['HV_neg_count_neg']               = HV_neg_BoW_count_answers_neg
answers_BoW['sentiment_score_answers_neg_HV'] = HV_sentiment_score_answers_neg

Processing Answers: 100%|██████████| 2877/2877 [14:46<00:00,  3.24it/s]


---
Approach 3: Create expanded dictionary by using word2vec
---

In [35]:
# merge the presentation and answers
merged_ec_global_w2v = pd.merge(
    EC_pres_BoW,
    answers_BoW[['file_name', 'QA_text_clean_non_neg','QA_text_clean_neg']], # Include 'QA_text_clean_non_neg' and 'QA_text_clean_neg' during the merge
    on='file_name',
    how='outer'
)

# merge the presentation and QA for word2vec without negations
merged_ec_global_w2v['combined_text_non_neg'] = merged_ec_global_w2v.progress_apply(lambda row: row['presentation_clean_non_neg'] + row['QA_text_clean_non_neg'], axis=1)

# merge the presentation adn QA for word2vec with negations
merged_ec_global_w2v['combined_text_neg'] = merged_ec_global_w2v.progress_apply(lambda row: row['presentation_clean_neg'] + row['QA_text_clean_neg'], axis=1) # Include 'QA_text_clean_non_neg' and 'QA_text_clean_neg' during the merge

100%|██████████| 2877/2877 [00:00<00:00, 5522.24it/s]


In [36]:
# create preprocessing function for word2vec
def preprocess_for_w2v(text):
    # If the text is a list, join the tokens back into a single string.
    if isinstance(text, list):
        text = " ".join(text)
    # Lowercase and strip whitespace
    text = text.lower().strip()
    # Split text into sentences
    sentences = sent_tokenize(text)
    tokenized_sentences = []
    for sentence in sentences:
        # Remove punctuation
        sentence = re.sub(r'[^\w\s]', '', sentence)
        # Tokenize sentence into words
        tokens = word_tokenize(sentence)
        tokenized_sentences.append(tokens)
    return tokenized_sentences


In [37]:
# Preprocess the text without negations
train_sentences_no_neg = []
for doc in tqdm(merged_ec_global_w2v['combined_text_non_neg'], desc="Processing texts without negations"):
    tokenized = preprocess_for_w2v(doc)
    train_sentences_no_neg.extend(tokenized)

# take a look
print(train_sentences_no_neg[:3])


Processing texts without negations: 100%|██████████| 2877/2877 [00:55<00:00, 52.22it/s]

[['lady', 'gentleman', 'thank', 'standing', 'welcome', 'first', 'quarter', 'quarterly', 'earnings', 'call', 'operator', 'instruction', 'reminder', 'conference', 'recorded', 'would', 'like', 'turn', 'conference', 'host', 'mr', 'mark', 'finkelstein', 'please', 'go', 'ahead', 'thank', 'christie', 'good', 'morning', 'thank', 'joining', 'call', 'representing', 'prudential', 'today', 'call', 'john', 'strangfeld', 'ceo', 'mark', 'grier', 'vice', 'chairman', 'charlie', 'lowery', 'head', 'international', 'business', 'steve', 'pelletier', 'head', 'domestic', 'business', 'rob', 'falzon', 'chief', 'financial', 'officer', 'rob', 'axel', 'controller', 'principal', 'accounting', 'officer', 'start', 'prepared', 'comment', 'john', 'mark', 'rob', 'answer', 'question', 'today', 'presentation', 'may', 'include', 'forward', 'looking', 'statement', 'possible', 'actual', 'result', 'may', 'differ', 'materially', 'prediction', 'make', 'today', 'addition', 'presentation', 'may', 'include', 'reference', 'non', '

In [38]:
# train word2vec model on the text without negations
w2v_model_no_neg = Word2Vec(sentences=train_sentences_no_neg, vector_size=100, window=5, min_count=3, workers=4)
w2v_model_no_neg.save("w2v_model_no_neg.model")

In [39]:
# Create the seed from LM without negations
seed_pos_no_neg_LM = LM_pos_BoW
seed_neg_no_neg_LM = LM_neg_BoW

# create the seed from HV witouht negations
seed_pos_no_neg_HV = HV_pos_BoW
seed_neg_no_neg_HV = HV_neg_BoW

In [40]:
# create a function to expand dictionary
def expand_dictionary(seed_words, model, topn=2, similarity_threshold=0.95):
    """
    Expand a dictionary based on seed words using Word2Vec.
    """
    expanded_words = set(seed_words)  # Start with the original seeds.
    for word in seed_words:
        if word in model.wv:
            try:
                similar_words = model.wv.most_similar(word, topn=topn)
            except KeyError:
                similar_words = []
            for similar_word, similarity in similar_words:
                if similarity >= similarity_threshold:
                    expanded_words.add(similar_word)
    return list(expanded_words)

In [41]:
# expand the dictionaries without negations for LM
expanded_pos_no_neg_LM = expand_dictionary(seed_pos_no_neg_LM, w2v_model_no_neg, 2, 0.95)
expanded_neg_no_neg_LM = expand_dictionary(seed_neg_no_neg_LM, w2v_model_no_neg, 2, 0.95)

# take a look at the added words
new_words_no_neg_LM = set(expanded_pos_no_neg_LM + expanded_neg_no_neg_LM) - set(seed_pos_no_neg_LM + seed_neg_no_neg_LM)
print("New positive and negative words added:", new_words_no_neg_LM)


New positive and negative words added: set()


In [42]:
# expand the dictionaires without neg for HV
expanded_pos_no_neg_HV = expand_dictionary(seed_pos_no_neg_HV, w2v_model_no_neg, 2, 0.95)
expanded_neg_no_neg_HV = expand_dictionary(seed_neg_no_neg_HV, w2v_model_no_neg, 2, 0.95)

# take a look at the added words
new_words_no_neg_HV = set(expanded_pos_no_neg_HV + expanded_neg_no_neg_HV) - set(seed_pos_no_neg_HV + seed_neg_no_neg_HV)
print("New positive and negative words added:", new_words_no_neg_HV)

New positive and negative words added: {'feast', 'investorgoodyearcom', 'leatherby', 'meiers'}


**Calculate sentiment scores using expanded dictionaries without negations**

In [43]:
# takes approximatly 3.30 min to run
# create dicts to store counts
LM_pos_Bow_count_pres_exp   = []
LM_neg_BoW_count_pres_exp   = []
LM_sentiment_score_pres_exp  = []

#loop over the presentations to find positive and negative words
for words in tqdm(EC_pres_BoW['presentation_clean_non_neg'], total = len(EC_pres_BoW['presentation_clean_non_neg']), desc = "Processing Presentations"):

  pos_count_lm_BoW  = sum(word in expanded_pos_no_neg_LM for word in words)
  neg_count_lm_BoW  = sum(word in expanded_neg_no_neg_LM for word in words)

  LM_pos_Bow_count_pres_exp.append(pos_count_lm_BoW)
  LM_neg_BoW_count_pres_exp.append(neg_count_lm_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_lm_BoW - neg_count_lm_BoW) / total_words if total_words > 0 else 0
  LM_sentiment_score_pres_exp.append(sentiment_score)

EC_pres_BoW['LM_pos_count']              = LM_pos_Bow_count_pres_exp
EC_pres_BoW['LM_neg_count']              = LM_neg_BoW_count_pres_exp
EC_pres_BoW['sentiment_score_pres_LM_exp']   = LM_sentiment_score_pres_exp


Processing Presentations: 100%|██████████| 2877/2877 [03:08<00:00, 15.29it/s]


In [44]:
# takes approximatly  min to run
# create dicts to store counts
LM_pos_Bow_count_answers_exp        = []
LM_neg_BoW_count_answers_exp        = []
LM_sentiment_score_answers_exp      = []

#loop over the answers to find positive and negative words
for words in tqdm(answers_BoW['QA_text_clean_non_neg'], total=len(answers_BoW['QA_text_clean_non_neg']), desc="Processing Answers"):

  pos_count_lm_BoW  = sum(word in expanded_pos_no_neg_LM for word in words)
  neg_count_lm_BoW  = sum(word in expanded_neg_no_neg_LM for word in words)

  LM_pos_Bow_count_answers_exp.append(pos_count_lm_BoW)
  LM_neg_BoW_count_answers_exp.append(neg_count_lm_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_lm_BoW - neg_count_lm_BoW) / total_words if total_words > 0 else 0
  LM_sentiment_score_answers_exp.append(sentiment_score)

answers_BoW['LM_pos_count']                   = LM_pos_Bow_count_answers_exp
answers_BoW['LM_neg_count']                   = LM_neg_BoW_count_answers_exp
answers_BoW['sentiment_score_answers_LM_exp'] = LM_sentiment_score_answers

Processing Answers: 100%|██████████| 2877/2877 [03:07<00:00, 15.34it/s]


In [45]:
# takes approximatly 6min to run
# create dicts to store counts
HV_pos_Bow_count_pres_exp    = []
HV_neg_BoW_count_pres_exp    = []
HV_sentiment_score_pres_exp  = []

#loop over the presentations to find positive and negative words
for words in tqdm(EC_pres_BoW['presentation_clean_non_neg'], total = len(EC_pres_BoW['presentation_clean_non_neg']), desc = "Processing Presentations"):

  pos_count_HV_BoW_exp  = sum(word in HV_pos_BoW for word in words)
  neg_count_HV_BoW_exp  = sum(word in HV_neg_BoW for word in words)

  HV_pos_Bow_count_pres_exp.append(pos_count_HV_BoW_exp)
  HV_neg_BoW_count_pres_exp.append(neg_count_HV_BoW_exp)

  total_words       = len(words)

  sentiment_score         = (pos_count_HV_BoW_exp - neg_count_HV_BoW_exp) / total_words if total_words > 0 else 0
  HV_sentiment_score_pres_exp.append(sentiment_score)

EC_pres_BoW['HV_pos_count']              = HV_pos_Bow_count_pres_exp
EC_pres_BoW['HV_neg_count']              = HV_neg_BoW_count_pres_exp
EC_pres_BoW['sentiment_score_pres_HV_exp']   = HV_sentiment_score_pres_exp


Processing Presentations: 100%|██████████| 2877/2877 [05:48<00:00,  8.26it/s]


In [46]:
# takes approximtley 6.30 min to run
# create dicts to store counts for HV
HV_pos_BoW_count_answers_exp        = []
HV_neg_BoW_count_answers_exp        = []
HV_sentiment_score_answers_exp      = []

#loop over the answers to find positive and negative words
for words in tqdm(answers_BoW['QA_text_clean_non_neg'], total=len(answers_BoW['QA_text_clean_non_neg']), desc="Processing Answers"):

  pos_count_HV_BoW  = sum(word in HV_pos_BoW for word in words)
  neg_count_HV_BoW  = sum(word in HV_neg_BoW for word in words)

  HV_pos_BoW_count_answers_exp.append(pos_count_HV_BoW)
  HV_neg_BoW_count_answers_exp.append(neg_count_HV_BoW)

  total_words       = len(words)

  sentiment_score         = (pos_count_HV_BoW - neg_count_HV_BoW) / total_words if total_words > 0 else 0
  HV_sentiment_score_answers_exp.append(sentiment_score)

answers_BoW['HV_pos_count']               = HV_pos_BoW_count_answers_exp
answers_BoW['HV_neg_count']               = HV_neg_BoW_count_answers_exp
answers_BoW['sentiment_score_answers_HV_exp'] = HV_sentiment_score_answers_exp

Processing Answers: 100%|██████████| 2877/2877 [05:48<00:00,  8.25it/s]


**Preprocess the text with negations**



In [47]:
# preprocess the text with negations
train_sentences_neg = []
for doc in tqdm(merged_ec_global_w2v['combined_text_neg'], desc="Processing texts with negations"):
    tokenized = preprocess_for_w2v(doc)
    train_sentences_neg.extend(tokenized)

# take a look
print(train_sentences_neg[:3])

Processing texts with negations: 100%|██████████| 2877/2877 [02:42<00:00, 17.65it/s]

[['ladies', 'and', 'gentlemen', 'thank', 'you', 'for', 'standing', 'by', 'and', 'welcome', 'to', 'the', 'first', 'quarter', '2016', 'quarterly', 'earnings', 'call'], ['operator', 'instructions', 'as', 'a', 'reminder', 'this', 'conference', 'is', 'being', 'recorded'], ['i', 'would', 'now', 'like', 'to', 'turn', 'the', 'conference', 'over', 'to', 'your', 'host', 'mr', 'mark', 'finkelstein', 'please', 'go', 'ahead']]


In [48]:
# train word2vec model on the text with negations
w2v_model_neg = Word2Vec(sentences=train_sentences_neg, vector_size=100, window=5, min_count=3, workers=4)
w2v_model_neg.save("w2v_model_neg.model")


In [49]:
# create the seed from LM with negations
seed_pos_neg_LM = LM_pos_neg_BoW
seed_neg_neg_LM = LM_neg_neg_BoW

# create the seed from HV with negations
seed_pos_neg_HV = HV_pos_neg_BoW
seed_neg_neg_HV = HV_neg_neg_BoW

In [50]:
# expand the dicitonaries with negations for LM
expanded_pos_neg_LM = expand_dictionary(seed_pos_neg_LM, w2v_model_neg, 1, 0.95)
expanded_neg_neg_LM = expand_dictionary(seed_neg_neg_LM, w2v_model_neg, 1, 0.95)

# expand the dict with negations for HV
expanded_pos_neg_HV = expand_dictionary(seed_pos_neg_HV, w2v_model_neg, 1, 0.95)
expanded_neg_neg_HV = expand_dictionary(seed_neg_neg_HV, w2v_model_neg, 1, 0.95)

# take a look at the added words
new_words_neg_LM = set(expanded_pos_neg_LM + expanded_neg_neg_LM) - set(seed_pos_neg_LM + seed_neg_neg_LM)
print("New positive and negative words added:", new_words_neg_LM)

new_words_neg_HV = set(expanded_pos_neg_HV + expanded_neg_neg_HV) - set(seed_pos_neg_HV + seed_neg_neg_HV)
print("New positive and negative words added:", new_words_neg_HV)

New positive and negative words added: set()
New positive and negative words added: {'feast', 'meiers', 'rancher', 'steininger'}


**Calcualte new sentiment scores form the expanded dictionaries with negations**

In [51]:
# takes approximatly 8 min to run
# inialize the lists
LM_pos_Bow_count_pres_neg_exp   = []
LM_neg_BoW_count_pres_neg_exp   = []
LM_sentiment_score_pres_neg_exp    = []

#loop over the presentations to find positive and negative words
for text in tqdm(EC_pres_BoW['presentation_clean_neg'], total=len(EC_pres_BoW['presentation_clean_neg']), desc = "Processing presentations with negations"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in expanded_pos_neg_LM:
        neg_count += 1
      elif base_word in expanded_neg_neg_LM:
        pos_count += 1
    else:
      if word in expanded_pos_neg_LM:
        pos_count += 1
      elif word in expanded_neg_neg_LM:
        neg_count += 1

  LM_pos_Bow_count_pres_neg_exp.append(pos_count)
  LM_neg_BoW_count_pres_neg_exp.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  LM_sentiment_score_pres_neg_exp.append(sentiment_score)

EC_pres_BoW['LM_pos_count_neg']              = LM_pos_Bow_count_pres_neg_exp
EC_pres_BoW['LM_neg_count_neg']              = LM_neg_BoW_count_pres_neg_exp
EC_pres_BoW['sentiment_score_pres_neg_LM_exp']  = LM_sentiment_score_pres_neg_exp

Processing presentations with negations: 100%|██████████| 2877/2877 [08:03<00:00,  5.94it/s]


In [52]:
# takes approximatly 10 min to run
# intilze the lists
LM_pos_Bow_count_answers_neg_exp        = []
LM_neg_BoW_count_answers_neg_exp        = []
LM_sentiment_score_answers_neg_exp      = []

#loop over the answers to find positive and negative words
for text in tqdm(answers_BoW['QA_text'], total=len(answers_BoW['QA_text']), desc = "Processing Answers"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in expanded_pos_neg_LM:
        neg_count += 1
      elif base_word in expanded_neg_neg_LM:
        pos_count += 1
    else:
      if word in expanded_pos_neg_LM:
        pos_count += 1
      elif word in expanded_neg_neg_LM:
        neg_count += 1

  LM_pos_Bow_count_answers_neg_exp.append(pos_count)
  LM_neg_BoW_count_answers_neg_exp.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  LM_sentiment_score_answers_neg_exp.append(sentiment_score)

answers_BoW['LM_pos_count_neg']               = LM_pos_Bow_count_answers_neg_exp
answers_BoW['LM_neg_count_neg']               = LM_neg_BoW_count_answers_neg_exp
answers_BoW['sentiment_score_answers_neg_LM_exp'] = LM_sentiment_score_answers_neg_exp

Processing Answers: 100%|██████████| 2877/2877 [10:02<00:00,  4.78it/s]


In [53]:
# takes approximatley 13 min to run
# inialize the lists
HV_pos_Bow_count_pres_neg_exp   = []
HV_neg_BoW_count_pres_neg_exp   = []
HV_sentiment_score_pres_neg_exp = []

#loop over the presentations to find positive and negative words
for text in tqdm(EC_pres_BoW['presentation_clean_neg'], total=len(EC_pres_BoW['presentation_clean_neg']), desc = "Processing presentations with negations"):

  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in expanded_pos_neg_HV:
        neg_count += 1
      elif base_word in expanded_neg_neg_HV:
        pos_count += 1
    else:
      if word in expanded_pos_neg_HV:
        pos_count += 1
      elif word in expanded_neg_neg_HV:
        neg_count += 1

  HV_pos_Bow_count_pres_neg_exp.append(pos_count)
  HV_neg_BoW_count_pres_neg_exp.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  HV_sentiment_score_pres_neg_exp.append(sentiment_score)

EC_pres_BoW['HV_pos_count_neg']              = HV_pos_Bow_count_pres_neg_exp
EC_pres_BoW['HV_neg_count_neg']              = HV_neg_BoW_count_pres_neg_exp
EC_pres_BoW['sentiment_score_pres_neg_HV_exp']   = HV_sentiment_score_pres_neg_exp

Processing presentations with negations: 100%|██████████| 2877/2877 [12:28<00:00,  3.84it/s]


In [54]:
# takes apporximatley 15 min to run
# intilze the lists
HV_pos_Bow_count_answers_neg_exp        = []
HV_neg_BoW_count_answers_neg_exp        = []
HV_sentiment_score_answers_neg_exp      = []

#loop over the answers to find positive and negative words
for text in tqdm(answers_BoW['QA_text'], total=len(answers_BoW['QA_text']), desc = "Processing Answers"):
  words       = word_tokenize(text)
  words_neg   = mark_negation(words)

  pos_count = 0
  neg_count = 0

  for word in words_neg:
    if '_NEG' in word:
      base_word = word.rstrip('_NEG')
      if base_word in expanded_pos_neg_HV:
        neg_count += 1
      elif base_word in expanded_neg_neg_HV:
        pos_count += 1
    else:
      if word in expanded_pos_neg_HV:
        pos_count += 1
      elif word in expanded_neg_neg_HV:
        neg_count += 1

  HV_pos_Bow_count_answers_neg_exp.append(pos_count)
  HV_neg_BoW_count_answers_neg_exp.append(neg_count)

  total_words       = len([w for w in words if w.isalpha()])
  sentiment_score = (pos_count - neg_count) / total_words if total_words > 0 else 0
  HV_sentiment_score_answers_neg_exp.append(sentiment_score)

answers_BoW['HV_pos_count_neg']               = HV_pos_Bow_count_answers_neg_exp
answers_BoW['HV_neg_count_neg']               = HV_neg_BoW_count_answers_neg_exp
answers_BoW['sentiment_score_answers_neg_HV_exp'] = HV_sentiment_score_answers_neg_exp

Processing Answers: 100%|██████████| 2877/2877 [14:53<00:00,  3.22it/s]


---
**Standardize scores and perpare for regressions**
---

In [55]:
# standardize for presentation LM
scaler_pres_BoW_LM = StandardScaler()

EC_pres_BoW['sentiment_score_pres_std_LM']        = scaler_pres_BoW_LM.fit_transform(EC_pres_BoW[['sentiment_score_pres_LM']])
EC_pres_BoW['sentiment_score_pres_neg_std_LM']    = scaler_pres_BoW_LM.fit_transform(EC_pres_BoW[['sentiment_score_pres_neg_LM']])

# standardie for presentation HV
scaler_pres_BoW_HV = StandardScaler()

EC_pres_BoW['sentiment_score_pres_std_HV']        = scaler_pres_BoW_HV.fit_transform(EC_pres_BoW[['sentiment_score_pres_HV']])
EC_pres_BoW['sentiment_score_pres_neg_std_HV']    = scaler_pres_BoW_HV.fit_transform(EC_pres_BoW[['sentiment_score_pres_neg_HV']])

# standardize for answers LM
scaler_answers_BoW_LM = StandardScaler()

answers_BoW['sentiment_score_answers_std_LM']      = scaler_answers_BoW_LM.fit_transform(answers_BoW[['sentiment_score_answers_LM']])
answers_BoW['sentiment_score_answers_neg_std_LM']  = scaler_answers_BoW_LM.fit_transform(answers_BoW[['sentiment_score_answers_neg_LM']])

# standardize for answers HV
scaler_answers_BoW_HV = StandardScaler()

answers_BoW['sentiment_score_answers_std_HV']      = scaler_answers_BoW_HV.fit_transform(answers_BoW[['sentiment_score_answers_HV']])
answers_BoW['sentiment_score_answers_neg_std_HV']  = scaler_answers_BoW_HV.fit_transform(answers_BoW[['sentiment_score_answers_neg_HV']])

In [56]:
# Standardize for presentation LM (expanded)
scaler_pres_BoW_LM_exp = StandardScaler()
EC_pres_BoW['sentiment_score_pres_std_LM_exp'] = scaler_pres_BoW_LM_exp.fit_transform(EC_pres_BoW[['sentiment_score_pres_LM_exp']])
EC_pres_BoW['sentiment_score_pres_neg_std_LM_exp'] = scaler_pres_BoW_LM_exp.fit_transform(EC_pres_BoW[['sentiment_score_pres_neg_LM_exp']])

# Standardize for presentation HV (expanded)
scaler_pres_BoW_HV_exp = StandardScaler()
EC_pres_BoW['sentiment_score_pres_std_HV_exp'] = scaler_pres_BoW_HV_exp.fit_transform(EC_pres_BoW[['sentiment_score_pres_HV_exp']])
EC_pres_BoW['sentiment_score_pres_neg_std_HV_exp'] = scaler_pres_BoW_HV_exp.fit_transform(EC_pres_BoW[['sentiment_score_pres_neg_HV_exp']])

# Standardize for answers LM (expanded)
scaler_answers_BoW_LM_exp = StandardScaler()
answers_BoW['sentiment_score_answers_std_LM_exp'] = scaler_answers_BoW_LM_exp.fit_transform(answers_BoW[['sentiment_score_answers_LM_exp']])
answers_BoW['sentiment_score_answers_neg_std_LM_exp'] = scaler_answers_BoW_LM_exp.fit_transform(answers_BoW[['sentiment_score_answers_neg_LM_exp']])

# Standardize for answers HV (expanded)
scaler_answers_BoW_HV_exp = StandardScaler()
answers_BoW['sentiment_score_answers_std_HV_exp'] = scaler_answers_BoW_HV_exp.fit_transform(answers_BoW[['sentiment_score_answers_HV_exp']])
answers_BoW['sentiment_score_answers_neg_std_HV_exp'] = scaler_answers_BoW_HV_exp.fit_transform(answers_BoW[['sentiment_score_answers_neg_HV_exp']])

In [57]:
# take a look
EC_pres_BoW.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,sentiment_score_pres_neg_LM_exp,sentiment_score_pres_neg_HV_exp,sentiment_score_pres_std_LM,sentiment_score_pres_neg_std_LM,sentiment_score_pres_std_HV,sentiment_score_pres_neg_std_HV,sentiment_score_pres_std_LM_exp,sentiment_score_pres_neg_std_LM_exp,sentiment_score_pres_std_HV_exp,sentiment_score_pres_neg_std_HV_exp
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,0.002751,0.015754,-1.332424,-1.195693,-0.055023,-0.031424,-1.332424,-1.195693,-0.055023,-0.031515
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,0.004250,0.016250,-1.156602,-1.014589,-0.192013,0.045910,-1.156602,-1.014589,-0.192013,0.045821
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,0.014841,0.022262,-0.118174,0.264754,0.623192,0.983163,-0.118174,0.264754,0.623192,0.983104
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,0.002099,0.009797,-1.262029,-1.274367,-1.013272,-0.960079,-1.262029,-1.274367,-1.013272,-0.960199
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,0.007197,0.009526,-0.529606,-0.658582,-0.998017,-1.002363,-0.529606,-0.658582,-0.998017,-1.002485


In [58]:
# take a look
answers_BoW.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,sentiment_score_answers_neg_LM_exp,sentiment_score_answers_neg_HV_exp,sentiment_score_answers_std_LM,sentiment_score_answers_neg_std_LM,sentiment_score_answers_std_HV,sentiment_score_answers_neg_std_HV,sentiment_score_answers_std_LM_exp,sentiment_score_answers_neg_std_LM_exp,sentiment_score_answers_std_HV_exp,sentiment_score_answers_neg_std_HV_exp
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,0.013291,0.013688,0.982097,0.974764,1.352354,0.978984,0.982097,0.974764,1.352354,0.978991
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,0.008449,0.010139,0.155597,0.066551,0.340495,0.153735,0.155597,0.066551,0.340495,0.153787
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,0.008533,0.008939,0.236519,0.082365,-0.154579,-0.125077,0.236519,0.082365,-0.154579,-0.125009
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,0.001823,0.005673,-1.032803,-1.176132,-1.008498,-0.884673,-1.032803,-1.176132,-1.008498,-0.884564
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,0.009048,0.010364,-0.080934,0.178851,0.041131,0.206048,-0.080934,0.178851,0.041131,0.206098


In [60]:
# save to sql

# Convert list columns to strings before saving to SQL using apply
for col in ['presentation_clean_non_neg', 'QA_text_clean_non_neg', 'presentation_clean_neg', 'QA_text_clean_neg','combined_text_non_neg','combined_text_neg']:
    if col in EC_pres_BoW.columns:
        EC_pres_BoW[col] = EC_pres_BoW[col].apply(lambda x: str(x) if isinstance(x, list) else x)

for col in ['QA_text_clean_non_neg', 'QA_text_clean_neg']:
    if col in answers_BoW.columns:
        answers_BoW[col] = answers_BoW[col].apply(lambda x: str(x) if isinstance(x, list) else x)



EC_pres_BoW.to_sql('EC_pres_BoW', conn, if_exists='replace', index=False)
answers_BoW.to_sql('answers_BoW', conn, if_exists='replace', index=False)
conn.close()

In [61]:
# save the sql files to google drive
from google.colab import drive
import time
# Mount Google Drive
drive.mount('/content/drive')

# Create a timestamped filename, e.g., LLMs_results_20250409-142500.db
timestamp = time.strftime("%Y%m%d-%H%M%S")
destination = f"/content/drive/My Drive/LLMs_results_{timestamp}.db"


Mounted at /content/drive


In [62]:
# save the sql files to google drive
!cp /content/LLMs_results.db "{destination}"